<a href="https://colab.research.google.com/github/MinamiNaoya/ExperimentTools/blob/main/leash_tutorial_ecfps_and_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Leash Tutorial - ECFPs and Random Forest
## Introduction

There are many ways to represent molecules for machine learning.

In this tutorial we will go through one of the simplest: ECFPs [[1]](https://pubs.acs.org/doi/10.1021/ci100050t) and Random Forest. This technique is surprisingly powerful, and on previous benchmarks often gets uncomfortably close to the state of the art.

First molecule graphs are broken into bags of subgraphs of varying sizes.

![ecfp featurizing process (chemaxon)](https://docs.chemaxon.com/display/docs/images/download/attachments/1806333/ecfp_generation.png)

Then the bag of subgraphs is hashed into a bit vector

![hashing process (chemaxon)](https://docs.chemaxon.com/display/docs/images/download/attachments/1806333/ecfp_folding.png)

This can be thought of as analogous to the [hashing trick](https://en.wikipedia.org/wiki/Feature_hashing) [[2]](https://alex.smola.org/papers/2009/Weinbergeretal09.pdf) on bag of words for NLP problems, from the days before transformers.

RDKit, an open-source cheminformatics tool, is used for generating ECFP features. It facilitates the creation of hashed bit vectors, streamlining the process. We can install it as follows:

In [ ]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 39.7 MB/s eta 0:00:00


The training set is pretty big, but we can treat the parquet files as databases using duckdb. We will use this to sample down to a smaller dataset for demonstration purposes. Lets install duckdb as well.

In [ ]:
!pip install duckdb

In [ ]:
!pip install kaggle

from google.colab import drive
drive.mount('/content/drive')

import os
import json
f = open("/content/drive/MyDrive/kaggle.json", 'r')
json_data = json.load(f)
os.environ['KAGGLE_USERNAME'] = json_data['username']
os.environ['KAGGLE_KEY'] = json_data['key']

Mounted at /content/drive


In [ ]:
!kaggle competitions download -c leash-BELKA

100% 4.16G/4.16G [04:26<00:00, 15.9MB/s]
100% 4.16G/4.16G [04:26<00:00, 16.8MB/s]


In [ ]:
!unzip '/content/leash-BELKA.zip'

Archive:  /content/leash-BELKA.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: test.parquet            
  inflating: train.csv               
  inflating: train.parquet           


## Data Preparation

The training and testing data paths are defined for the .parquet files. We use duckdb to scan search through the large training sets. Just to get started lets sample out an equal number of positive and negatives.

This query selects an equal number of samples where binds equals 0 (non-binding) and 1 (binding), limited to 30,000 each, to avoid model bias towards a particular class.

トレーニングデータとテストデータのパスは.parquetファイルに対して定義されます。duckdbを使用して、大規模なトレーニングセットをスキャン検索します。まずは、ポジティブとネガティブを同数ずつ抽出します。このクエリでは、モデルが特定のクラスに偏らないように、bindsが0（非結合）と1（結合）の同数のサンプルを、それぞれ30,000個に制限して選択します。

In [ ]:
import duckdb
import pandas as pd

train_path = '/content/train.parquet'
test_path = '/content/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 40000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 40000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## sEH
エポキシドヒドロラーゼ2はEPHX2遺伝子座にコードされており、そのタンパク質産物は一般に「可溶性エポキシドヒドロラーゼ」、またはsEHと略称される。ヒドロラーゼは特定の化学反応を触媒する酵素であり、EPHX2/sEHもまた特定のリン酸基を加水分解する。EPHX2/sEHは、高血圧と糖尿病進行のための潜在的な薬物標的であり、以前のDELの努力からEPHX2/sEHを阻害する低分子が臨床試験に進んだ。
## BRD4
ブロモドメイン4はBRD4遺伝子座にコードされており、そのタンパク質産物もBRD4と命名されている。ブロモドメインは、DNAが巻き付く核内のタンパク質スプール（ヒストンと呼ばれる）に結合し、近くのDNAが転写される可能性に影響を与え、新しい遺伝子産物を作り出す。ブロモドメインは癌の進行に関与しており、その活性を阻害する薬剤が数多く発見されている。
## ALB
第3の標的である血清アルブミンはALB遺伝子座にコードされており、そのタンパク質産物もALBと命名されている。このタンパク質産物は「ヒト血清アルブミン」を意味するHSAと略されることもある。血液中で最も一般的なタンパク質であるALBは、浸透圧（組織から血管内に体液を戻す）を促進し、多くのリガンド、ホルモン、脂肪酸などを輸送するのに使われる。我々は、Active Motif社から購入したALBを審査した。タンパク質の構造情報を応募に取り入れたい応募者のために、アミノ酸配列はUniProtエントリーP02768の25位から609位、結晶構造はPDBエントリー1AO6、予測構造はAlphaFold2エントリーP02768にあります。リガンドが結合したその他のALB結晶構造はPDBにある。

In [ ]:
df

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,283410838,O=C(O)[C@@H]1C[C@@H]2CCCC[C@@H]2N1C(=O)OCC1c2c...,CN1CC(CN)CC1=O,Nc1nc(Cl)c2[nH]cnc2n1,CN1CC(CNc2nc(Nc3nc(Cl)c4[nH]cnc4n3)nc(N3[C@H](...,HSA,0
1,113653839,O=C(N[C@H](C(=O)O)C1CCCCC1)OCC1c2ccccc2-c2ccccc21,Cc1cc(F)ccc1N,NCC1(c2ccc(Cl)cc2Cl)CCCC1,Cc1cc(F)ccc1Nc1nc(NCC2(c3ccc(Cl)cc3Cl)CCCC2)nc...,BRD4,0
2,75787238,O=C(NC1(C(=O)O)CC1)OCC1c2ccccc2-c2ccccc21,Cl.NCCNC(=O)c1ccccc1F,Br.NCc1cccc(Br)n1,O=C(NCCNc1nc(NCc2cccc(Br)n2)nc(NC2(C(=O)N[Dy])...,sEH,0
3,33864313,COc1ccc(C(=O)O)c(NC(=O)OCC2c3ccccc3-c3ccccc32)c1,CCC(CC)(CN)OC,Cl.NC[C@@H]1CCCO1,CCC(CC)(CNc1nc(NC[C@@H]2CCCO2)nc(Nc2cc(OC)ccc2...,HSA,0
4,153194860,O=C(Nc1c(I)c(C(=O)O)c(I)c(C(=O)O)c1I)OCC1c2ccc...,Cl.NCc1nnc2c(=O)[nH]ccn12,Cn1cc(CN)cn1,Cn1cc(CNc2nc(NCc3nnc4c(=O)[nH]ccn34)nc(Nc3c(I)...,HSA,0
...,...,...,...,...,...,...,...
79995,18777072,CC(OC(C)(C)C)C(NC(=O)OCC1c2ccccc2-c2ccccc21)C(...,COc1ccc(CN)c(C)c1OC,CC1(C)CC(CN)C(C)(C)O1,COc1ccc(CNc2nc(NCC3CC(C)(C)OC3(C)C)nc(NC(C(=O)...,BRD4,1
79996,197694383,O=C(Nc1ccc(Cl)c(C(=O)O)c1)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1ccc2ccccc2n1,Cc1cc2cc(CN)ccc2[nH]1,Cc1cc2cc(CNc3nc(NCc4ccc5ccccc5n4)nc(Nc4ccc(Cl)...,sEH,1
79997,248278912,O=C(O)C[C@@H](Cc1ccc(Cl)cc1Cl)NC(=O)OCC1c2cccc...,Nc1ccc2nccnc2c1,Cc1cc2cc(CN)ccc2[nH]1,Cc1cc2cc(CNc3nc(Nc4ccc5nccnc5c4)nc(N[C@@H](CC(...,HSA,1
79998,82684805,O=C(NCC1CCC(C(=O)O)CC1)OCC1c2ccccc2-c2ccccc21,Nc1cccc(-n2cncn2)c1,Nc1ccc2c(c1)OCCCO2,O=C(N[Dy])C1CCC(CNc2nc(Nc3cccc(-n4cncn4)c3)nc(...,sEH,1


### PDBファイルのダウンロード

In [ ]:
!pip install biopython
import time
import urllib
from Bio.PDB import PDBList

pdb_ids = ['1ao6', '7jkz', '3i1y']

def download_file(url, dst_path):
    with urllib.request.urlopen(url) as web_file:
        with open(dst_path, 'wb') as local_file:
            local_file.write(web_file.read())

# AlphaFold
url = "https://alphafold.ebi.ac.uk/files/AF-P02768-F1-model_v4.pdb"

dst_path = "AF-P02768-F1-model_v4.pdb"
download_file(url, dst_path)

pdbl = PDBList()

for pdb_id in pdb_ids:
    pdbl.retrieve_pdb_file(pdb_id, pdir='pdb_files/')
    time.sleep(10)




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.6 MB/s eta 0:00:00


### OpenMMの利用

In [ ]:
!wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local
!rm Miniconda3-latest-Linux-x86_64.sh

PREFIX=/usr/local
Unpacking payload ...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /usr/local


In [ ]:
!conda create -n myenv -y

Channels:
 - defaults
Platform: linux-64
Solving environment: / done

## Package Plan ##

  environment location: /usr/local/envs/myenv



Preparing transaction: \ done
Verifying transaction: / - \ done
Executing transaction: / done
#
# To activate this environment, use
#
#     $ conda activate myenv
#
# To deactivate an active environment, use
#
#     $ conda deactivate



In [ ]:
!conda update conda -y
!conda init bash
!source activate myenv

Channels:
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - conda


The following packages will be SUPERSEDED by a higher-priority channel:

  conda              conda-forge::conda-24.5.0-py312h7900f~ --> pkgs/main::conda-24.5.0-py312h06a4308_0 




Preparing transaction: / done
Verifying transaction: \ done
Executing transaction: / done
no change     /usr/local/condabin/conda
no change     /usr/local/bin/conda
no change     /usr/local/bin/conda-env
no change     /usr/local/bin/activate
no change     /usr/local/bin/deactivate
no change     /usr/local/etc/profile.d/conda.sh
no change     /usr/local/etc/fish/conf.d/conda.fish
no change     /usr/local/shell/condabin/Conda.psm1
no change     /usr/local/shell/condabin/conda-hook.ps1
no change     /usr/local/lib/python3.12/site-packages/xontrib/conda.xsh
no change     /usr/local/etc/profile.d/conda.csh
no change     /ro

In [ ]:
!conda install -c conda-forge openmm -y

Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - openmm


The following packages will be SUPERSEDED by a higher-priority channel:

  conda              pkgs/main::conda-24.5.0-py312h06a4308~ --> conda-forge::conda-24.5.0-py312h7900ff3_0 




Preparing transaction: / done
Verifying transaction: \ done
Executing transaction: / - done


In [ ]:
!conda install -c conda-forge openmm-setup -y

Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: - \ done

# All requested packages already installed.



In [ ]:
!conda install -c conda-forge pdbfixer -y

Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: / - done

# All requested packages already installed.



In [ ]:
import os
os.environ['PATH'] = '/usr/local/miniconda/bin:' + os.environ['PATH']

In [ ]:
import openmm as mm
import openmm.app as app
from openmm import unit
import pdbfixer

def preprocessing(pdb_file):
  # PDBFixerでPDBファイルを読み込む
  fixer = pdbfixer.PDBFixer(pdb_file)

  # 分子力場の設定
  forcefield = app.ForceField("amber14-all.xml", "amber14/tip3pfb.xml")

  # 不要な構造の削除
  fixer.removeHeterogens()

  # 欠けている残基のチェック（欠損原子の確認のためにも必要）
  fixer.findMissingResidues()

  # タンパク質末端の欠けている残基を取り除く処理
  chains = list(fixer.topology.chains())
  keys = fixer.missingResidues.keys()
  for key in list(keys):
      chain = chains[key[0]]
      if key[1] == 0 or key[1] == len(list(chain.residues())):
          del fixer.missingResidues[key]

  # 非標準な残基が含まれているか確認、あれば標準的なものに置き換える
  fixer.findNonstandardResidues()
  fixer.replaceNonstandardResidues()

  # 欠けている原子の確認、あれば追加する
  fixer.findMissingAtoms()
  fixer.addMissingAtoms()

  # 水素原子の付与（pHを設定する）
  ph = 7.0
  fixer.addMissingHydrogens(ph)

  # 水ボックスの追加（力場、paddingの厚み、イオン濃度（デフォルトはNaCl））
  modeller = app.Modeller(fixer.topology, fixer.positions)
  modeller.addSolvent(forcefield, padding=1.0 * unit.nanometers, ionicStrength=0.15 * unit.molar)

  # 処理後の状態（トポロジー、原子の位置）をPDBファイルで出力
  top = modeller.getTopology()
  pos = modeller.getPositions()
  app.PDBFile.writeFile(top, pos, open(f'Data/processed_{pdb_file.replace(".pdb", "")}.pdb', 'w'))

## Feature Preprocessing

Lets grab the smiles for the fully assembled molecule `molecule_smiles` and generate ecfps for it. We could choose different radiuses or bits, but 2 and 1024 is pretty standard.

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder

# Convert SMILES to RDKit molecules
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

# Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule'].apply(generate_ecfp)

## Train Model

In [ ]:
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the random forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")




Mean Average Precision (mAP): 0.97


Look at that Average Precision score. We did amazing!

Actually no, we just overfit. This is likely recurring theme for this competition. It is easy to predict molecules that come from the same corner of chemical space, but generalizing to new molecules is extremely difficult.

## Test Prediction

 The trained Random Forest model is then used to predict the binding probabilities. These predictions are saved to a CSV file, which serves as the submission file for the Kaggle competition.

In [ ]:
import os

# Process the test.parquet file chunk by chunk
test_file = '/content/test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file

# Read the test.parquet file into a pandas DataFrame
for df_test in pd.read_csv(test_file, chunksize=100000):

    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

    # One-hot encode the protein_name
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

    # Predict the probabilities
    probabilities = rf_model.predict_proba(X_test)[:, 1]

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))
